In [31]:
import numpy as np
import pandas_ta as ta
import pandas as pd
from datetime import datetime, timedelta, timezone
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [35]:
df_min = pd.read_csv('vn30f1m_3min.csv')
df_hour = pd.read_csv('vn30f1m_1hour.csv')
df_day = pd.read_csv('vn30f1m_1day.csv')

In [36]:
def data_backtest(data):
    new_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    new_data.index = pd.to_datetime(data['Date'])
    new_data.index.name = None
    return new_data

In [37]:
def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

In [48]:
def process_data(df):
    data = df.copy()
    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    
    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values

    return data_model

In [38]:
data_min = data_backtest(df_min)
data_hour = data_backtest(df_hour)
data_day = data_backtest(df_day)

In [40]:
data_min

,Open,High,Low,Close,Volume
2018-08-13 09:00:00,943.5,943.6,942.9,943.5,1316
2018-08-13 09:03:00,943.3,943.4,942.9,943.0,839
2018-08-13 09:06:00,943.1,943.5,943.1,943.3,790
2018-08-13 09:09:00,943.2,943.3,943.1,943.2,529
2018-08-13 09:12:00,943.0,943.1,942.6,943.1,868
...,...,...,...,...,...
2024-08-30 14:21:00,1332.3,1333.3,1332.3,1332.7,2734
2024-08-30 14:24:00,1332.9,1334.0,1332.9,1333.5,3396
2024-08-30 14:27:00,1333.3,1333.3,1332.0,1332.3,4926
2024-08-30 14:30:00,1332.3,1332.3,1332.3,1332.3,196


In [39]:
train_min, valid_min = split_data(data_min)
train_hour, valid_hour = split_data(data_hour)
train_day, valid_day = split_data(data_day)

In [ ]:
def indicator(data):
    # Hàm indicator
    formular

    return something

class CustomStrategy(Strategy):

    def init(self):
        # Note that this returns each column as a row
        # Then extends the length of each row on each iteration
        self.something = self.I(indicator, self.data)

    def next(self):

        sth1 = self.something[0]
        sth2 = self.something[2]

        if self.position:
            if something happens:
                self.position.close()
        else:
            if something happens:
                self.buy()
                
bt = Backtest(data, CustomStrategy, cash=3000, commission=0.01, margin=0.13, hedging=True, exclusive_orders=False)
stats = bt.run()
stats